# 📊 Évaluation Comparative des Architectures Conversationnelles

Ce notebook présente une structure d'évaluation complète pour les trois architectures expérimentales développées pour EasyTransfert. Il décrit les métriques communes d'évaluation et fournit un cadre pour comparer les performances de chaque architecture.

## 🎯 Objectifs de l'Évaluation

1. **Quantifier les performances** de chaque architecture
2. **Identifier les forces et faiblesses** de chaque approche
3. **Guider la sélection** de l'architecture optimale selon le contexte
4. **Établir des benchmarks** pour les améliorations futures

<a href="https://colab.research.google.com/github/AmedBah/memoire/blob/main/notebooks/evaluation/04_evaluation_comparative_architectures.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 🚀 Configuration pour Google Colab

**Note**: Cette section est spécifique à Google Colab. Si vous exécutez ce notebook localement, vous pouvez ignorer ces cellules.

In [ ]:
# Vérifier le type de runtime
import os
import sys

# Détecter si on est sur Colab
IS_COLAB = 'google.colab' in sys.modules

if IS_COLAB:
    print("✓ Exécution sur Google Colab")
    
    # Vérifier le type de GPU
    import torch
    if torch.cuda.is_available():
        gpu_name = torch.cuda.get_device_name(0)
        print(f"✓ GPU détecté: {gpu_name}")
        print(f"✓ Mémoire GPU: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB")
        
        # Recommandations selon le GPU
        if 'T4' in gpu_name:
            print("\n⚠️  GPU T4 détecté (16 GB): Convient pour ce notebook mais les temps d'entraînement seront plus longs")
        elif 'V100' in gpu_name or 'A100' in gpu_name:
            print(f"\n✓ {gpu_name}: Parfait pour ce notebook!")
    else:
        print("\n❌ ATTENTION: Aucun GPU détecté!")
        print("   Pour activer le GPU: Runtime > Change runtime type > GPU")
        print("   Pour Colab Pro: Choisir 'High-RAM' ou 'Premium GPU'")
else:
    print("✓ Exécution locale")
    import torch
    if torch.cuda.is_available():
        print(f"✓ GPU disponible: {torch.cuda.get_device_name(0)}")
    else:
        print("⚠️  Aucun GPU détecté - l'entraînement sera très lent")

In [ ]:
# Monter Google Drive (uniquement sur Colab)
if IS_COLAB:
    from google.colab import drive
    drive.mount('/content/drive')
    
    # Définir le chemin vers les données
    # OPTION 1: Données dans Google Drive
    # DATA_DIR = '/content/drive/MyDrive/memoire/data'
    
    # OPTION 2: Cloner le repo et utiliser les données locales
    print("\n📥 Clonage du repository...")
    !git clone https://github.com/AmedBah/memoire.git /content/memoire
    DATA_DIR = '/content/memoire/data'
    
    print(f"\n✓ Répertoire de données: {DATA_DIR}")
    
    # Vérifier que les données sont présentes
    if os.path.exists(DATA_DIR):
        print("✓ Données trouvées!")
        !ls -lh {DATA_DIR}
    else:
        print(f"❌ ERREUR: Répertoire {DATA_DIR} non trouvé!")
        print("   Veuillez soit:")
        print("   1. Copier le dossier 'data' dans votre Google Drive")
        print("   2. Ou le repository sera cloné automatiquement")
else:
    # Exécution locale
    DATA_DIR = '../../data'
    print(f"✓ Répertoire de données local: {DATA_DIR}")

In [ ]:
# Fonction helper pour obtenir les chemins de données
def get_data_path(relative_path):
    """Obtenir le chemin absolu d'un fichier de données"""
    return os.path.join(DATA_DIR, relative_path)

# Exemples de chemins
print("Chemins de données configurés:")
print(f"  Conversations: {get_data_path('conversations/conversation_1000_finetune.jsonl')}")
print(f"  FAQs: {get_data_path('faqs/faq_easytransfert.json')}")
print(f"  Opérateurs: {get_data_path('operators/operators_info.json')}")
print(f"  Procédures: {get_data_path('procedures/procedures_resolution.json')}")
print(f"  Expressions: {get_data_path('expressions/expressions_ivoiriennes.json')}")
print(f"  Documents: {get_data_path('documents/doc.txt.txt')}")

## 📋 Table des Métriques d'Évaluation

Les métriques sont organisées en trois catégories principales :

1. **Métriques Techniques** : Performance système et ressources
2. **Métriques de Qualité** : Qualité des réponses et pertinence
3. **Métriques Métier** : Impact sur le service client

## 🔧 Métriques Techniques

Les métriques techniques évaluent les performances système et l'utilisation des ressources.

### 1. Latence (Temps de Réponse)

**Description :** Temps écoulé entre la soumission d'une requête et la réception de la réponse complète.

**Unité :** Millisecondes (ms) ou Secondes (s)

**Calcul :**
```python
latence = temps_fin_réponse - temps_début_requête
```

**Importance :** Une faible latence améliore l'expérience utilisateur et permet de traiter plus de requêtes.

**Benchmarks par architecture :**
- **Architecture 1** : ~2-3 secondes
- **Architecture 2** : ~3-5 secondes (150-200ms récupération + 2-3s génération)
- **Architecture 3** : ~4-7 secondes (overhead agentique)

**Métriques dérivées :**
- Latence moyenne (mean)
- Latence médiane (median)
- Latence percentile 95 (p95)
- Latence percentile 99 (p99)
- Écart-type (std)

---

### 2. Throughput (Débit)

**Description :** Nombre de requêtes traitées par unité de temps.

**Unité :** Requêtes par seconde (req/s)

**Calcul :**
```python
throughput = nombre_requêtes / temps_total
```

**Importance :** Indique la capacité de l'architecture à gérer la charge.

**Benchmarks attendus :**
- **Architecture 1** : ~0.3-0.5 req/s (latence faible)
- **Architecture 2** : ~0.2-0.3 req/s
- **Architecture 3** : ~0.15-0.25 req/s (plus complexe)

---

### 3. Utilisation Mémoire

**Description :** Quantité de mémoire RAM et VRAM utilisée pendant l'exécution.

**Unité :** Mégaoctets (MB) ou Gigaoctets (GB)

**Composants mesurés :**
- **RAM** : Modèle, données, cache
- **VRAM** : Modèle sur GPU, activations
- **Stockage** : Base vectorielle (ChromaDB)

**Benchmarks par architecture :**
- **Architecture 1** : ~50 MB (adaptateurs LoRA seuls)
- **Architecture 2** : ~150-300 MB (base vectorielle + modèle)
- **Architecture 3** : ~200-400 MB (outils + agent + base)

---

### 4. Coût Computationnel

**Description :** Ressources de calcul nécessaires (CPU/GPU).

**Métriques :**
- **FLOPs** : Opérations en virgule flottante
- **Utilisation GPU** : Pourcentage d'utilisation
- **Coût par requête** : Coût infrastructure par requête

**Importance :** Détermine le coût opérationnel et la scalabilité.

## 🎯 Métriques de Qualité

Les métriques de qualité évaluent la pertinence et la fiabilité des réponses générées.

### 1. Pertinence des Réponses

**Description :** Degré auquel la réponse est appropriée et répond à la question posée.

**Méthodes d'évaluation :**

#### a) Évaluation Humaine
- Échelle de 1 à 5
- Critères : cohérence, complétude, utilité
- Nécessite annotateurs formés

#### b) Similarité Sémantique
```python
# Cosine similarity entre réponse et référence
from sentence_transformers import SentenceTransformer, util

model = SentenceTransformer('paraphrase-multilingual-mpnet-base-v2')
embedding_reponse = model.encode(reponse_generee)
embedding_reference = model.encode(reponse_reference)
score_pertinence = util.cos_sim(embedding_reponse, embedding_reference)
```

#### c) BERTScore
- Utilise embeddings contextuels
- Plus robuste que BLEU/ROUGE
- Score de précision, rappel, F1

**Seuils d'acceptabilité :**
- Excellent : > 0.85
- Bon : 0.70 - 0.85
- Acceptable : 0.50 - 0.70
- Insuffisant : < 0.50

---

### 2. Factualité et Hallucinations

**Description :** Mesure la véracité des informations fournies et détecte les inventions (hallucinations).

**Taux d'hallucination :**
```python
taux_hallucination = (nombre_reponses_avec_erreurs / nombre_total_reponses) * 100
```

**Types d'hallucinations :**
1. **Intrinsèques** : Information contradictoire avec la source
2. **Extrinsèques** : Information non vérifiable dans les sources

**Méthodes de détection :**
- Vérification contre base de connaissances
- Cross-référencement avec sources citées
- Validation par API de données opérationnelles

**Benchmarks attendus :**
- **Architecture 1** : Taux élevé (~20-40%) - pas de sources
- **Architecture 2** : Taux moyen (~5-15%) - RAG mais pas de validation
- **Architecture 3** : Taux faible (~2-8%) - validation par outils

---

### 3. Complétude (Couverture des Informations)

**Description :** Proportion des informations pertinentes incluses dans la réponse.

**Calcul :**
```python
completude = (elements_informatifs_presents / elements_informatifs_attendus) * 100
```

**Éléments évalués :**
- Toutes les étapes d'une procédure mentionnées
- Informations contextuelles (frais, délais, limites)
- Avertissements ou précautions

**Exemple - Transfert MTN vers Orange :**
- ✅ Montant min/max
- ✅ Frais de transaction
- ✅ Format de l'identifiant
- ✅ Délai de traitement
- ✅ Étapes de la procédure

---

### 4. Traçabilité (Citation des Sources)

**Description :** Capacité à fournir et référencer les sources d'information.

**Métriques :**
- **Taux de citation** : % de réponses avec sources
- **Précision des sources** : Sources citées sont correctes
- **Pertinence des sources** : Sources citées sont pertinentes

**Calcul :**
```python
taux_citation = (reponses_avec_sources / total_reponses) * 100
precision_sources = (sources_correctes / sources_citees) * 100
```

**Benchmarks par architecture :**
- **Architecture 1** : 0% (aucune traçabilité)
- **Architecture 2** : ~80-95% (sources RAG)
- **Architecture 3** : ~90-100% (cycle ReAct transparent)

---

### 5. Cohérence et Consistance

**Description :** Les réponses à des questions similaires sont cohérentes entre elles.

**Test :** Poser la même question reformulée plusieurs fois

**Calcul :**
```python
# Similarité entre N réponses à la même question
from itertools import combinations
import numpy as np

similarities = []
for r1, r2 in combinations(reponses, 2):
    sim = calculate_similarity(r1, r2)
    similarities.append(sim)
    
score_coherence = np.mean(similarities)
```

**Seuil acceptable :** > 0.75

---

### 6. Adaptation Linguistique

**Description :** Capacité à comprendre et utiliser les expressions locales (ivoiriennes).

**Test :** Requêtes avec expressions ivoiriennes
- "Mon go n'est pas arrivé" (mon argent)
- "Dèh, je veux cotiser" (transaction de groupe)
- "C'est combien le gbê?" (quel est le prix/tarif)

**Métriques :**
- Taux de compréhension : % requêtes comprises
- Utilisation appropriée dans les réponses

## 💼 Métriques Métier (Business)

Les métriques métier évaluent l'impact sur le service client et les objectifs d'entreprise.

### 1. Taux de Résolution au Premier Contact

**Description :** Proportion de requêtes résolues sans nécessiter d'intervention humaine ou d'escalade.

**Calcul :**
```python
taux_resolution = (requetes_resolues / total_requetes) * 100
```

**Critères de résolution :**
- Réponse complète et correcte
- Client satisfait (pas de relance)
- Pas de transfert vers agent humain

**Objectifs :**
- **Architecture 1** : ~40-60% (limitée aux cas simples)
- **Architecture 2** : ~60-75% (meilleure fiabilité)
- **Architecture 3** : ~75-90% (gestion cas complexes)

---

### 2. Score de Satisfaction Client

**Description :** Mesure la satisfaction des utilisateurs avec les réponses reçues.

**Méthodes de collecte :**
1. **Feedback explicite** : Pouce haut/bas après chaque réponse
2. **CSAT (Customer Satisfaction Score)** : Échelle 1-5
3. **NPS (Net Promoter Score)** : Probabilité de recommandation

**Calcul CSAT :**
```python
csat = (notes_4_et_5 / total_reponses) * 100
```

**Benchmarks :**
- Excellent : CSAT > 80%
- Bon : CSAT 60-80%
- À améliorer : CSAT < 60%

---

### 3. Taux d'Escalade (Transfert vers Agent Humain)

**Description :** Proportion de conversations transférées à un agent humain.

**Calcul :**
```python
taux_escalade = (conversations_escaladees / total_conversations) * 100
```

**Raisons d'escalade :**
- Agent ne peut pas résoudre
- Demande client explicite
- Cas complexe ou sensible
- Problème technique

**Objectifs :**
- **Architecture 1** : ~40-60% (limitations importantes)
- **Architecture 2** : ~25-40% (meilleure couverture)
- **Architecture 3** : ~10-25% (autonomie maximale)

---

### 4. Temps Moyen de Résolution

**Description :** Durée moyenne pour résoudre complètement une requête client.

**Calcul :**
```python
temps_moyen = (somme_durees_conversations / nombre_conversations_resolues)
```

**Composants :**
- Temps de latence système
- Nombre d'échanges nécessaires
- Temps de lecture client

**Objectifs :**
- Réponse immédiate : < 1 minute
- Résolution simple : 2-3 minutes
- Résolution complexe : 5-10 minutes

---

### 5. Taux de Containment

**Description :** Capacité de l'assistant à gérer les requêtes de bout en bout sans sortir du système.

**Calcul :**
```python
taux_containment = ((total_conversations - escalades - abandons) / total_conversations) * 100
```

**Importance :** Réduction des coûts opérationnels

---

### 6. Réduction de la Charge des Agents

**Description :** Pourcentage de requêtes gérées par l'assistant vs agents humains.

**Impact :**
- **Réduction coûts** : Moins d'agents nécessaires
- **Disponibilité 24/7** : Service continu
- **Agents focalisés** : Cas complexes uniquement

**Calcul ROI :**
```python
economies_annuelles = (requetes_automatisees * cout_requete_agent) - cout_infrastructure
roi = (economies_annuelles / cout_developpement) * 100
```

## 🔬 Méthodologie d'Évaluation Comparative

### Dataset de Test

Pour une évaluation juste, utiliser un ensemble de test standardisé :

**Composition recommandée :**
- **100-200 requêtes** couvrant tous les cas d'usage
- **Distribution :**
  - 40% : Questions simples (FAQ)
  - 30% : Requêtes procédurales (how-to)
  - 20% : Cas complexes (multi-étapes)
  - 10% : Cas edge/difficiles

**Catégories de requêtes :**
1. Informations générales (horaires, limites, frais)
2. Procédures de transfert
3. Résolution de problèmes
4. Vérification de statut
5. Questions sur opérateurs spécifiques

### Protocole d'Évaluation

**Étapes :**
1. Exécuter chaque requête sur les 3 architectures
2. Enregistrer toutes les métriques
3. Faire évaluer les réponses par annotateurs
4. Calculer les statistiques agrégées
5. Analyser les résultats par catégorie

**Conditions de test :**
- Même environnement hardware
- Même version des modèles
- Même base de connaissances
- Mesures répétées (3-5 fois) pour stabilité

## 📊 Framework de Comparaison

### Tableau Comparatif des Architectures

| Métrique | Architecture 1 | Architecture 2 | Architecture 3 | Objectif |
|----------|----------------|----------------|----------------|---------|
| **TECHNIQUES** | | | | |
| Latence moyenne | ~2-3s | ~3-5s | ~4-7s | < 5s |
| Throughput | ~0.3-0.5 req/s | ~0.2-0.3 req/s | ~0.15-0.25 req/s | > 0.2 req/s |
| Mémoire (RAM) | ~50 MB | ~150-300 MB | ~200-400 MB | < 500 MB |
| **QUALITÉ** | | | | |
| Pertinence | 0.65-0.75 | 0.75-0.85 | 0.80-0.90 | > 0.80 |
| Taux hallucination | 20-40% | 5-15% | 2-8% | < 10% |
| Complétude | 60-70% | 75-85% | 85-95% | > 80% |
| Traçabilité | 0% | 80-95% | 90-100% | > 80% |
| **MÉTIER** | | | | |
| Taux résolution | 40-60% | 60-75% | 75-90% | > 70% |
| CSAT | 60-70% | 70-80% | 80-90% | > 75% |
| Taux escalade | 40-60% | 25-40% | 10-25% | < 30% |
| Temps résolution | 2-3 min | 3-5 min | 3-7 min | < 5 min |

## 💻 Implémentation : Structure de Code pour l'Évaluation

Voici un exemple de structure de code pour implémenter l'évaluation comparative.

### 1. Installation des Dépendances

In [ ]:
# Installation des bibliothèques nécessaires
!pip install numpy pandas matplotlib seaborn
!pip install sentence-transformers bert-score
!pip install scikit-learn

### 2. Dataset de Test Standardisé

In [ ]:
# Dataset de test avec requêtes représentatives
test_queries = [
    # Questions simples (FAQ)
    {"query": "Bonjour, c'est quoi EasyTransfert ?", "category": "general", "complexity": "simple"},
    {"query": "Quels sont les frais de transaction ?", "category": "general", "complexity": "simple"},
    {"query": "Quelle est la limite de transfert MTN ?", "category": "operator", "complexity": "simple"},
    {"query": "Les horaires de service client ?", "category": "general", "complexity": "simple"},
    
    # Requêtes procédurales
    {"query": "Comment faire un transfert de MTN vers Orange ?", "category": "procedure", "complexity": "medium"},
    {"query": "Comment réinitialiser mon mot de passe ?", "category": "procedure", "complexity": "medium"},
    {"query": "Étapes pour créer un compte ?", "category": "procedure", "complexity": "medium"},
    
    # Résolution de problèmes
    {"query": "Mon transfert n'est pas arrivé, que faire ?", "category": "troubleshooting", "complexity": "complex"},
    {"query": "J'ai envoyé au mauvais numéro, puis-je annuler ?", "category": "troubleshooting", "complexity": "complex"},
    {"query": "Erreur: numéro invalide, comment corriger ?", "category": "troubleshooting", "complexity": "complex"},
    
    # Vérification statut (Architecture 3)
    {"query": "Vérifier le statut du transfert EFB.ABC123456", "category": "status", "complexity": "medium"},
    {"query": "Mon argent est où ? Ref: EFB.XYZ789012", "category": "status", "complexity": "medium"},
    
    # Expressions ivoiriennes
    {"query": "Dèh, mon go n'est pas arrivé", "category": "local", "complexity": "medium"},
    {"query": "C'est combien le gbê pour Orange ?", "category": "local", "complexity": "simple"},
    
    # Cas complexes
    {"query": "Différence entre transfert direct et via EasyTransfert ? Lequel est moins cher ?", "category": "complex", "complexity": "complex"},
    {"query": "Puis-je transférer 500000 FCFA de MTN vers Wave un dimanche soir ?", "category": "complex", "complexity": "complex"},
]

print(f"Dataset de test : {len(test_queries)} requêtes")
print(f"Distribution par complexité:")
for complexity in ['simple', 'medium', 'complex']:
    count = len([q for q in test_queries if q['complexity'] == complexity])
    print(f"  - {complexity.capitalize()}: {count} ({count/len(test_queries)*100:.1f}%)")

### 3. Classe d'Évaluation des Métriques

In [ ]:
import time
import numpy as np
from sentence_transformers import SentenceTransformer, util
from typing import Dict, List

class ArchitectureEvaluator:
    """Classe pour évaluer les performances d'une architecture"""
    
    def __init__(self, architecture_name: str):
        self.architecture_name = architecture_name
        self.embedding_model = SentenceTransformer('paraphrase-multilingual-mpnet-base-v2')
        self.results = []
        
    def evaluate_query(self, query: str, chat_function, reference_answer: str = None) -> Dict:
        """Évalue une requête unique"""
        
        # Métriques techniques
        start_time = time.time()
        response = chat_function(query)
        latency = (time.time() - start_time) * 1000  # en ms
        
        result = {
            'query': query,
            'response': response,
            'latency_ms': latency,
            'response_length': len(response.split()),
        }
        
        # Métriques de qualité (si référence disponible)
        if reference_answer:
            emb_response = self.embedding_model.encode(response, convert_to_tensor=True)
            emb_reference = self.embedding_model.encode(reference_answer, convert_to_tensor=True)
            similarity = util.cos_sim(emb_response, emb_reference).item()
            result['relevance_score'] = similarity
            
        self.results.append(result)
        return result
    
    def evaluate_dataset(self, test_queries: List, chat_function, num_runs: int = 3):
        """Évalue l'ensemble du dataset de test"""
        
        print(f"\n🔍 Évaluation de {self.architecture_name}")
        print("=" * 80)
        
        for idx, test_case in enumerate(test_queries):
            query = test_case['query']
            print(f"\nRequête {idx+1}/{len(test_queries)}: {query[:60]}...")
            
            # Exécuter plusieurs fois pour stabilité
            latencies = []
            for run in range(num_runs):
                result = self.evaluate_query(query, chat_function)
                latencies.append(result['latency_ms'])
            
            print(f"  Latence: {np.mean(latencies):.0f}ms (±{np.std(latencies):.0f}ms)")
            
        return self.compute_aggregate_metrics()
    
    def compute_aggregate_metrics(self) -> Dict:
        """Calcule les métriques agrégées"""
        
        latencies = [r['latency_ms'] for r in self.results]
        response_lengths = [r['response_length'] for r in self.results]
        
        metrics = {
            'architecture': self.architecture_name,
            'num_queries': len(self.results),
            
            # Métriques techniques
            'latency_mean': np.mean(latencies),
            'latency_median': np.median(latencies),
            'latency_std': np.std(latencies),
            'latency_p95': np.percentile(latencies, 95),
            'latency_p99': np.percentile(latencies, 99),
            'throughput': 1000 / np.mean(latencies),  # req/s
            
            # Métriques de réponse
            'response_length_mean': np.mean(response_lengths),
            'response_length_std': np.std(response_lengths),
        }
        
        # Métriques de qualité (si disponibles)
        relevance_scores = [r.get('relevance_score') for r in self.results if 'relevance_score' in r]
        if relevance_scores:
            metrics['relevance_mean'] = np.mean(relevance_scores)
            metrics['relevance_std'] = np.std(relevance_scores)
        
        return metrics
    
    def print_summary(self, metrics: Dict):
        """Affiche un résumé des métriques"""
        
        print(f"\n\n📊 RÉSUMÉ - {metrics['architecture'].upper()}")
        print("=" * 80)
        print(f"\n🔧 MÉTRIQUES TECHNIQUES:")
        print(f"  Latence moyenne    : {metrics['latency_mean']:.0f}ms (±{metrics['latency_std']:.0f}ms)")
        print(f"  Latence médiane    : {metrics['latency_median']:.0f}ms")
        print(f"  Latence P95        : {metrics['latency_p95']:.0f}ms")
        print(f"  Latence P99        : {metrics['latency_p99']:.0f}ms")
        print(f"  Throughput         : {metrics['throughput']:.2f} req/s")
        
        print(f"\n📝 MÉTRIQUES DE RÉPONSE:")
        print(f"  Longueur moyenne   : {metrics['response_length_mean']:.1f} mots (±{metrics['response_length_std']:.1f})")
        
        if 'relevance_mean' in metrics:
            print(f"\n🎯 MÉTRIQUES DE QUALITÉ:")
            print(f"  Pertinence moyenne : {metrics['relevance_mean']:.3f} (±{metrics['relevance_std']:.3f})")
        
        print("=" * 80)

### 4. Évaluation Comparative des 3 Architectures

In [ ]:
# Exemple d'utilisation pour comparer les 3 architectures
# NOTE: Remplacer par les vraies fonctions de chat de chaque architecture

# Simulation des fonctions de chat (à remplacer)
def chat_arch1(query):
    """Fonction de chat Architecture 1 (à connecter au vrai modèle)"""
    # return model_arch1.chat(query)
    time.sleep(2.5)  # Simulation latence
    return "Réponse simulée de l'Architecture 1"

def chat_arch2(query):
    """Fonction de chat Architecture 2 (à connecter au vrai modèle)"""
    # return rag_chat(query)
    time.sleep(4.0)  # Simulation latence
    return "Réponse simulée de l'Architecture 2 avec sources"

def chat_arch3(query):
    """Fonction de chat Architecture 3 (à connecter au vrai modèle)"""
    # return agentic_chat(query)
    time.sleep(5.5)  # Simulation latence
    return "Réponse simulée de l'Architecture 3 avec cycle ReAct"

# Créer les évaluateurs
evaluator1 = ArchitectureEvaluator("Architecture 1 - Fine-tuning Simple")
evaluator2 = ArchitectureEvaluator("Architecture 2 - RAG Standard")
evaluator3 = ArchitectureEvaluator("Architecture 3 - RAG-Agentique")

# Évaluer chaque architecture (utiliser subset pour test rapide)
test_subset = test_queries[:5]  # Pour test rapide, utiliser [:5]

print("\n" + "="*80)
print("🚀 DÉBUT DE L'ÉVALUATION COMPARATIVE")
print("="*80)

metrics1 = evaluator1.evaluate_dataset(test_subset, chat_arch1, num_runs=1)
evaluator1.print_summary(metrics1)

metrics2 = evaluator2.evaluate_dataset(test_subset, chat_arch2, num_runs=1)
evaluator2.print_summary(metrics2)

metrics3 = evaluator3.evaluate_dataset(test_subset, chat_arch3, num_runs=1)
evaluator3.print_summary(metrics3)

### 5. Visualisation Comparative

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Créer DataFrame comparatif
comparison_df = pd.DataFrame([
    metrics1,
    metrics2,
    metrics3
])

# Visualisations
fig, axes = plt.subplots(2, 2, figsize=(15, 10))
fig.suptitle('Comparaison des Performances des Architectures', fontsize=16, fontweight='bold')

# 1. Latence moyenne
axes[0, 0].bar(comparison_df['architecture'], comparison_df['latency_mean'], color=['#3498db', '#e74c3c', '#2ecc71'])
axes[0, 0].set_ylabel('Latence (ms)')
axes[0, 0].set_title('Latence Moyenne')
axes[0, 0].tick_params(axis='x', rotation=15)

# 2. Throughput
axes[0, 1].bar(comparison_df['architecture'], comparison_df['throughput'], color=['#3498db', '#e74c3c', '#2ecc71'])
axes[0, 1].set_ylabel('Requêtes/seconde')
axes[0, 1].set_title('Throughput (Débit)')
axes[0, 1].tick_params(axis='x', rotation=15)

# 3. Latence P95 et P99
x = range(len(comparison_df))
width = 0.35
axes[1, 0].bar([i - width/2 for i in x], comparison_df['latency_p95'], width, label='P95', color='#3498db')
axes[1, 0].bar([i + width/2 for i in x], comparison_df['latency_p99'], width, label='P99', color='#e74c3c')
axes[1, 0].set_ylabel('Latence (ms)')
axes[1, 0].set_title('Latence Percentiles')
axes[1, 0].set_xticks(x)
axes[1, 0].set_xticklabels(comparison_df['architecture'], rotation=15)
axes[1, 0].legend()

# 4. Longueur des réponses
axes[1, 1].bar(comparison_df['architecture'], comparison_df['response_length_mean'], color=['#3498db', '#e74c3c', '#2ecc71'])
axes[1, 1].set_ylabel('Nombre de mots')
axes[1, 1].set_title('Longueur Moyenne des Réponses')
axes[1, 1].tick_params(axis='x', rotation=15)

plt.tight_layout()
plt.savefig('comparison_architectures.png', dpi=300, bbox_inches='tight')
plt.show()

print("\n✅ Graphiques sauvegardés : comparison_architectures.png")

### 6. Export des Résultats

In [ ]:
# Exporter les résultats en CSV
comparison_df.to_csv('evaluation_results.csv', index=False)
print("\n✅ Résultats exportés : evaluation_results.csv")

# Afficher tableau comparatif
print("\n" + "="*80)
print("📊 TABLEAU COMPARATIF FINAL")
print("="*80)
print(comparison_df[['architecture', 'latency_mean', 'latency_median', 'throughput', 'response_length_mean']].to_string(index=False))
print("="*80)

## 🎯 Recommandations Finales

### Sélection de l'Architecture Selon le Contexte

#### Choisir Architecture 1 si :
- ✅ Budget limité / POC rapide
- ✅ Requêtes très simples et répétitives
- ✅ Besoin de latence ultra-faible
- ✅ Données d'entraînement de qualité disponibles
- ❌ Mais : Risque d'hallucinations élevé

#### Choisir Architecture 2 si :
- ✅ Besoin de fiabilité et traçabilité
- ✅ Base de connaissances évolutive
- ✅ Balance performance/complexité
- ✅ Réduction des hallucinations prioritaire
- ❌ Mais : Limité pour requêtes multi-étapes

#### Choisir Architecture 3 si :
- ✅ Service client complet et automatisé
- ✅ Requêtes complexes nécessitant raisonnement
- ✅ Besoin d'accès données opérationnelles
- ✅ Maximisation de l'autonomie
- ✅ Budget infrastructure suffisant
- ⚠️ Nécessite infrastructure robuste et maintenance

### Métriques Prioritaires par Objectif

**Si priorité = Coût :**
- Minimiser : Utilisation mémoire, coût computationnel
- → **Architecture 1**

**Si priorité = Fiabilité :**
- Maximiser : Factualité, traçabilité, pertinence
- → **Architecture 2 ou 3**

**Si priorité = Autonomie :**
- Maximiser : Taux résolution, taux containment
- Minimiser : Taux escalade
- → **Architecture 3**

**Si priorité = Performance :**
- Minimiser : Latence
- Maximiser : Throughput
- → **Architecture 1**

## 📚 Références et Ressources

### Métriques NLP
- **BLEU** : Papineni et al. (2002) - "BLEU: a Method for Automatic Evaluation of Machine Translation"
- **ROUGE** : Lin (2004) - "ROUGE: A Package for Automatic Evaluation of Summaries"
- **BERTScore** : Zhang et al. (2020) - "BERTScore: Evaluating Text Generation with BERT"

### Métriques RAG
- **RAGAS** : Framework d'évaluation RAG (Retrieval-Augmented Generation Assessment)
- **Faithfulness** : Mesure de fidélité aux sources
- **Answer Relevancy** : Pertinence de la réponse à la question

### Outils d'Évaluation
- **Sentence-Transformers** : Similarité sémantique
- **Hugging Face Evaluate** : Bibliothèque de métriques
- **MLflow** : Tracking d'expérimentations
- **Weights & Biases** : Monitoring et comparaison

### Benchmarks Conversationnels
- **DSTC** : Dialog System Technology Challenge
- **ConvAI** : Conversational Intelligence Challenge
- **MultiWOZ** : Multi-Domain Wizard-of-Oz dataset

## 📝 Notes Finales

### Bonnes Pratiques d'Évaluation

1. **Évaluation Continue** : Ne pas évaluer qu'une fois, mais régulièrement
2. **Données Réelles** : Utiliser des vraies requêtes clients quand possible
3. **Évaluation Humaine** : Combiner métriques automatiques et jugement humain
4. **A/B Testing** : Tester en production avec échantillons réels
5. **Monitoring** : Suivre métriques en temps réel post-déploiement

### Limitations de l'Évaluation Automatique

- Les métriques automatiques ne capturent pas toute la qualité
- L'évaluation humaine reste le gold standard
- Le contexte métier est crucial pour interpréter les résultats
- Les métriques doivent être adaptées au cas d'usage spécifique

### Prochaines Étapes

1. Adapter ce notebook aux architectures implémentées
2. Enrichir le dataset de test avec vraies requêtes EasyTransfert
3. Mettre en place évaluation humaine avec annotateurs
4. Intégrer tracking avec Weights & Biases ou MLflow
5. Déployer monitoring en production

---

## 📞 Contact et Support

Pour toute question sur l'évaluation ou les architectures :

- **Projet** : EasyTransfert - KAYBIC AFRICA
- **Documentation** : Voir `ARCHITECTURE_README.md`
- **Support** : 2522018730 (WhatsApp 24h/24)

---

*Ce notebook fait partie du projet de mémoire sur les systèmes conversationnels pour EasyTransfert*